In [39]:
import os
os.environ['KERAS_BACKEND'] = "tensorflow"  # actually tensorflow is the default backend. 
import numpy as np
from keras import utils
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import keras
from keras import layers
from random import randint
import sqlite3
import math
import pandas as pd
import tensorflow as tf

In [40]:
# Read in the data from the database
conn = sqlite3.connect('data/tables.db')
# show database content
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
# Extract two tables from it and store them in two pd df
ds = pd.read_sql_query("SELECT * from df_2023_h1_feature", conn)
target = pd.read_sql_query("SELECT * from df_2023_h1_target", conn)

[('df_2023_h1_feature',), ('df_2023_h1_target',)]


In [62]:
def seq(data, seq_length):
    xseq = []
    yseq = []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = target.iloc[i+seq_length]
        xseq.append(x)
        yseq.append(y)
    return np.array(xseq), np.array(yseq)
seq_length = 10 # Number of days to look back
X,y = seq(ds, seq_length)

In [63]:
X.shape

(331598, 10, 22)

In [64]:
# custom train test val split

X_train = X[0:10000]
y_train = y[0:10000]
X_val = X[10001:11001]
y_val = y[10001:11001]
X_test = X[11002: 12002]
y_test = y[11002: 12002]
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(10000, 10, 22) (10000, 1) (1000, 10, 22) (1000, 1) (1000, 10, 22) (1000, 1)


In [37]:
simpke_option_model = keras.Sequential([
    layers.InputLayer(22),
    layers.SimpleRNN(4),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

ValueError: Input 0 of layer "simple_rnn" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 22)

In [13]:
simple_option_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 features (InputLayer)       [(None, 22)]              0         
                                                                 
 dense (Dense)               (None, 32)                736       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 769 (3.00 KB)
Trainable params: 769 (3.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
simple_option_model.compile(optimizer = "adam",
              loss = keras.losses.MeanSquaredError,
              metrics=['accuracy']
)
simple_option_model.fit(ds_new, target_new)

TypeError: in user code:

    File "/Users/xipengdu/anaconda3/envs/PIC16B-24W/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/xipengdu/anaconda3/envs/PIC16B-24W/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/xipengdu/anaconda3/envs/PIC16B-24W/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/xipengdu/anaconda3/envs/PIC16B-24W/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/xipengdu/anaconda3/envs/PIC16B-24W/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/Users